In [6]:
#### Word embeddings
#### Setup

!pip install tf-nightly
import tensorflow as tf
import tensorflow_datasets as tfds
import os

In [7]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

811008/807992 [==============================] - 0s 0us/step


'C:\\Users\\Nano\\.keras\\datasets'

In [8]:
#### Load text into datasets
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [9]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [10]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [11]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'The Gods, that mighty labor, from beside'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'That flies before them, screaming; so those two,'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"Survey'd the walls of Troy, the ships of Greece,">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Jove gives us now a day worth all the past;'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'For Hector, who the thighs of fatted bulls'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [12]:
#### Encode text lines as numbers

tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

In [13]:
#### Encode examples

encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [14]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'The Gods, that mighty labor, from beside'


In [15]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[10336, 10635, 2054, 2450, 5207, 10617, 1096]


In [16]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

In [17]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

In [18]:
#### Split the dataset into test and train batches

train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))

In [19]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [20]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(15,), dtype=int64, numpy=
 array([10336, 10635,  2054,  2450,  5207, 10617,  1096,     0,     0,
            0,     0,     0,     0,     0,     0], dtype=int64)>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>)

In [21]:
vocab_size += 1

In [22]:
#### Build the model

model = tf.keras.Sequential()

In [23]:
#### The first layer converts integer representations to dense vector embeddings. 

model.add(tf.keras.layers.Embedding(vocab_size, 64))

In [24]:
#### The next layer is a Long Short-Term Memory layer, which lets the model understand words in their context with other words. 
#### A bidirectional wrapper on the LSTM helps it to learn about the datapoints in relationship to the datapoints that came before it and after it.

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [25]:
#### Finally we'll have a series of one or more densely connected layers, with the last one being the output layer. 
#### The output layer produces a probability for all the labels. 
#### The one with the highest probability is the models prediction of an example's label.

# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))

In [26]:
#### Finally, compile the model. 
#### For a softmax categorization model, use sparse_categorical_crossentropy as the loss function. 
#### You can try other optimizers, but adam is very common.

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [27]:
#### Train the model

model.fit(train_data, epochs=3, validation_data=test_data)

Instructions for updating:
Use `tf.data.Dataset.cardinality()
Epoch 1/3
697/697 [==============================] - 21s 31ms/step - loss: 0.5228 - accuracy: 0.7461 - val_loss: 0.3790 - val_accuracy: 0.8332
Epoch 2/3
697/697 [==============================] - 18s 25ms/step - loss: 0.3013 - accuracy: 0.8684 - val_loss: 0.3539 - val_accuracy: 0.8442
Epoch 3/3
697/697 [==============================] - 17s 25ms/step - loss: 0.2266 - accuracy: 0.9020 - val_loss: 0.3994 - val_accuracy: 0.8388


In [29]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

79/79 [==============================] - 2s 22ms/step - loss: 0.3994 - accuracy: 0.8388

Eval loss: 0.399, Eval accuracy: 0.839
